<a href="https://colab.research.google.com/github/kingjiwoo/nlpbible/blob/main/N_gram_%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8%EB%A1%9C_%EB%AC%B8%EC%9E%A5_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.0 MB/s eta 0:00:00


1. 실습명: N-gram 언어 모델로 문장 생성하기
2. 실습 목적 및 설명
- 파이썬의 NLTK 패키지를 이용하여 N-gram 언어 모델을 구축한다.
- 네이버에서 제공하는 nsmc 영화리뷰 데이터셋을 이용하여 문장을 생성한다.

3. flow
- 입력 텍스트를 N-gram으로 바꿈
- 단어의 빈도를 측정,
- 조건부 확률을 측정
- 단어열을 예측하고 생성하는 모델 구축

In [3]:
# import modules

import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import codecs
from konlpy.tag import Okt
from tqdm import tqdm

In [4]:
# NLTK 패키지를 이용하여 입력 텍스트를 N-gram 형태로 변환한다.
sentence = "나는 매일 아침 지하철을 탄다"

In [5]:
#NLTK 사용을 위해 선행 패키지를 설치한다.
nltk.download('punkt')

# 입력 텍스트를 띄어쓰기 기준으로 토큰화한다.
tokens = word_tokenize(sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# 토큰 확인
print(tokens)

['나는', '매일', '아침', '지하철을', '탄다']


In [7]:
# 한국어의 단어는 띄어쓰기를 기준으로 하지 않기 때문에 konlpy를 이용해 형태소를 기준으로 토큰화한다.

tagger = Okt()

def tokenize(text):
    tokens = [ '/'.join(t) for t in tagger.pos(text)]
    return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '지하철/Noun', '을/Josa', '탄다/Verb']


In [13]:
# 토큰을 N-gram의 형태로 바꾸어준다.
# ngrams 함수의 두 번째 인자로 N값을 지정할 수 있다.
bigram = ngrams(tokens, 2)
trigram = ngrams(tokens, 3)

In [14]:
# N-gram을 출력해본다.
print("bigram:")
for b in bigram:
    print(b)

print("\ntrigram:")
for t in trigram:
    print(t)

bigram:
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')

trigram:
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '지하철/Noun')
('아침/Noun', '지하철/Noun', '을/Josa')
('지하철/Noun', '을/Josa', '탄다/Verb')


In [15]:
# padding을 통해 입력 데이터에 문장의 시작과 끝을 알리는 토큰을 추가한다.
bigram = ngrams(tokens, 2, pad_left = True, pad_right= True, left_pad_symbol="<s>", right_pad_symbol="<\s>")
print("bigrams with padding:")
for b in bigram:
    print(b)

bigrams with padding:
('<s>', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '<\\s>')


In [16]:
# 문장 생성을 위하여 네이버 영화 리뷰 데이터셋을 다운로드한다.
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

CPU times: user 6.88 ms, sys: 0 ns, total: 6.88 ms
Wall time: 406 ms


In [17]:
#다운로드 받은 데이터셋을 읽고 인덱스와 라벨을 제외한 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다.
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # header 제외
print("데이터셋:", data[:10])

docs = [row[1] for row in data] # 텍스트 부분만가져옴
print("텍스트 데이터:", docs[:5])
print("문장 개수 ", len(docs))

데이터셋: [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]
텍스트 데이터: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']
문장 개수  150000


In [18]:
# 토큰화한 텍스트 데이터의 bigram을 모두 리스트에 추가한다.
sentences = []
for d in tqdm(docs):
    tokens = tokenize(d)
    bigram = ngrams(tokens, 2, pad_left = True, pad_right= True, left_pad_symbol="<s>", right_pad_symbol="<\s>")
    sentences += [t for t in bigram]

100%|██████████| 150000/150000 [08:42<00:00, 287.01it/s]


In [19]:
print(sentences[:10])

[('<s>', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', '<\\s>'), ('<s>', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]


In [20]:
# sentences에 대한 단어별 등장 빈도를 측정한다.
cfd = ConditionalFreqDist(sentences)

In [21]:
# "<s>" 다음에 가장 많이 오는 단어(즉, 문장 맨 처음에 가장 많이 오는 단어) 5개를 출력해본다.
print(cfd["<s>"].most_common(5))

[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069)]


In [24]:
# 주어진 토큰(c) 다음에 가장 많이 등장하는 n개의 단어를 반환하는 함수를 만든다.
def most_common(c, n, pos=None):
    if pos is None:
        return cfd[tokenize(c)[0]].most_common(n)
    else:
        return cfd["/".join([c, pos])].most_common(n)

print(most_common("나", 10))


[('는/Josa', 831), ('의/Josa', 339), ('만/Josa', 213), ('에게/Josa', 148), ('에겐/Josa', 84), ('랑/Josa', 81), ('한테/Josa', 50), ('참/Verb', 45), ('이/Determiner', 44), ('와도/Josa', 43)]


In [25]:
# 단어별 등장 빈도를 기반으로 조건부확률을 추정한다.
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [27]:
#"." 다음에 "</s>" 가 올 확률을 출력한다.
print(cpd[tokenize('.')[0]].prob("<\s>"))

0.39102658679807606


In [30]:
# 토큰 c 다음에 토큰 w가 bigram으로 함께 등장할 확률을 구한다.
def bigram_prob(c,w):
    context = tokenize(c)[0]
    word = tokenize(w)[0]
    return cpd[context].prob(word)

print(bigram_prob("영화", "이"))

0.00015767585785521414


In [33]:
# 조건부 확률을 알게되면 가장 확률이 높은 토큰열을 토대로 문장을 생성할 수 있다.
def generate_sentence (seed=None, debug=False):
    if seed is not None:
        import random
        random.seed(seed)
    c = "<s>"
    sentence = []
    while True:
        if c not in cpd:
            break
        w = cpd[c].generate()
        if w == "<\s>":
            break
        word = w.split("/")[0]
        pos = w.split("/")[1]

        # 조사, 어미 등을 제외하고 각 토큰은 띄어쓰기로 구분하여 생성한다.
        if c == "":
            sentence.append(word.title())
        elif c in ["`", "\"","'","("]:
            sentence.append(word)
        elif word in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence.append(word)
        elif pos in ["Josa", "Punctuation", "Suffix"]:
            sentence.append(word)
        elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                    "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
            sentence.append(word)
        else:
            sentence.append(" " + word)
        c = w

        if debug:
            print(w)

    return "".join(sentence)

print(generate_sentence(2))


 도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯
